In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
 # Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/daisyzhang/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


## NASA Mars News

In [3]:
url = "https://redplanetscience.com/"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# scraping the news title with tage and class name 
titles = soup.find_all('div', class_='content_title')

for news_title in titles:
    print('title:')
    print(news_title.text + "\n")

title:
HiRISE Views NASA's InSight and Curiosity on Mars

title:
What's Mars Solar Conjunction, and Why Does It Matter?

title:
NASA's Treasure Map for Water Ice on Mars

title:
NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound

title:
NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network

title:
Deadline Closing for Names to Fly on NASA's Next Mars Rover

title:
Global Storms on Mars Launch Dust Towers Into the Sky

title:
NASA's Perseverance Rover Mission Getting in Shape for Launch

title:
NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb

title:
Mars 2020 Stands on Its Own Six Wheels

title:
NASA Perseverance Mars Rover Scientists Train in the Nevada Desert

title:
NASA Adds Return Sample Scientists to Mars 2020 Leadership Team

title:
NASA's MAVEN Maps Winds in the Martian Upper Atmosphere that Mirror the Terrain Below and Gives Clues to Martian Climate

title:
NASA's MAVEN Observes Martian Night Sky Pulsing in Ultra

In [6]:
# scraping news paragraph 
paragraphs  = soup.find_all('div', class_='article_teaser_body')

for news_p in paragraphs:
    print('news body:')
    print(news_p.text + "\n")

news body:
New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.

news body:
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.

news body:
A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.

news body:
The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.

news body:
Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.

news body:
You have until Sept. 30 to send your names to Mars aboard the Mars 2020 rover. 

news body:
A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.

news body:
Stacking spacecraft

## JPL Mars Space Images - Featured Image

In [7]:
jpl_url = "https://spaceimages-mars.com/"
browser.visit(jpl_url)

In [8]:
html = browser.html
soup = bs(html, 'html.parser')

In [9]:
featured_image_url = soup.find('img',class_="headerimage fade-in")['src']

In [10]:
featured_image_url

'image/featured/mars3.jpg'

## Mars Facts

In [58]:
fact_url = "https://galaxyfacts-mars.com/"

In [59]:
# read from url
tables = pd.read_html(fact_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [60]:
fact_df=tables[0]
fact_df.head()

fact_df = fact_df.rename(columns={0:"Mars - Earth Comparison",1:"Mars",2:"Earth"})

In [61]:
fact_df = fact_df.drop([0, 0])
fact_df.set_index("Mars - Earth Comparison",inplace=True)

In [62]:
fact_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [65]:
# generate HTML tables from DataFrames
html_table = fact_df.to_html()
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Mars - Earth Comparison</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'